# Notebook for Serving The Trained Model for Classifying TinyImageNet. Notebook (4/4) in the End-to-End Scalable Deep Learning Pipeline on Hops.


This notebook will send inference requests to a model serving instance that was exported by notebook number three ([Notebook number three](./Step3_Distributed_Training.ipynb)). This assumes that you have created a model serving instance of the model by using the hopsworks UI. You can find documentation on how to do this [here](https://hops.readthedocs.io/en/0.9/hopsml/model_serving.html)

![step4.png](../images/step4.png)



## Imports

In [28]:
import requests
from hops import util
from hops import hdfs
from hops import constants
import os
from hops import featurestore
import tensorflow as tf

## Constants

In [29]:
PROJECT_ID = 21139
MODEL = "tinyimagenet_resnet50"
PROJECT = hdfs.project_name()
INFERENCE_URL = ("/" + 
                 constants.REST_CONFIG.HOPSWORKS_REST_RESOURCE + 
                 "/project/" + 
                 str(PROJECT_ID) + 
                 "/inference/models/" + 
                 MODEL + 
                 ":predict"
                )
SERVER = ""
WORK_DIR = ""
CONCURRENCY = ""
NUM_TESTS = ""
TEST_DATASET = "test_dataset_tinyimagenet"
HEIGHT = 64
WIDTH = 64
CHANNELS = 3
BATCH_SIZE = 100
SHUFFLE_BUFFER_SIZE = 1000
INPUT_SHAPE = 12288
NUM_CLASSES = 200

In [33]:
def get_tf_dataset():
    """
    Creates a Tensorflow Dataset from TFRecords on HopsFS stored in the feature store.
    
    Returns:
           Tensorflow dataset
    """
    
    # Get Path and Schema from feature store metadata
    tf_record_schema = featurestore.get_training_dataset_tf_record_schema(TEST_DATASET)
    dataset_dir = featurestore.get_training_dataset_path(TEST_DATASET)
    
    input_files = tf.gfile.Glob(dataset_dir + "/part-r-*")
    dataset = tf.data.TFRecordDataset(input_files)

    def decode(example_proto):
        example = tf.parse_single_example(example_proto, tf_record_schema)
        image_flat = example["image"]
        image = tf.reshape(image_flat, (HEIGHT,WIDTH,CHANNELS))
        return image

    dataset = dataset.map(decode)
    return dataset

In [34]:
def do_inference():
    with tf.Session() as sess:
        dataset = get_tf_dataset()
        dataset_iter = dataset.make_one_shot_iterator()
        for i in range(5):
            x = sess.run(dataset_iter.get_next())
            print("feature shape: {}".format(x.shape))
            request_data={}
            request_data['instances'] = x
            headers={}
            # TODO this will work in hopsworks 0.10.0 when JWT is enabled
            #headers[constants.HTTP_CONFIG.HTTP_AUTHORIZATION] = "Bearer " + util.get_jwt()
            #r = requests.post(INFERENCE_URL, headers=headers, data=json.dumps(request_data), verify=False)

In [36]:
#do_inference()